# Extreme gradient boosting with xgboost

**This is an online course provided by datacamp**

XGBoost (Extreme Gradient Boosting) belongs to a family of boosting algorithms and uses the gradient boosting framework at its core.

- optimized gradient-boosting machine learning library
- Originally written in C++

Why xgboost?

- speed and performance
- core algorithm is parallelizable
- consistently outperforms single-algorithm methods
- state-of-the-art performance in many ML task

you can use the scikit-learn `.fit()` / `.predict()` paradigm that you are already familiar to build your XGBoost models, as the xgboost library has a `scikit-learn` compatible API!

**Boosting** is a sequential technique which works on the principle of an ensemble. It combines a set of weak learners and delivers improved prediction accuracy. At any instant $t$, the model outcomes are weighted based on the outcomes of previous instant $t - 1$. The outcomes predicted correctly are given a lower weight and the ones miss-classified are weighted higher. Note that a **weak learner** is one which is slightly better than random guessing. 

In [1]:
import textwrap
import os
import sys
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn as sk
from sklearn.model_selection import train_test_split

print("Python version: {0}".format(sys.version))
print("pandas version: {0}".format(pd.__version__))
print("numpy version:{0}".format(np.__version__))
print("sklearn version:{0}".format(sk.__version__))
print("xgboost version: {0}".format(xgb.__version__))
print("Working directory: \n" + os.getcwd())

Python version: 3.7.1 (default, Dec 14 2018, 13:28:58) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
pandas version: 0.23.4
numpy version:1.15.4
sklearn version:0.20.1
xgboost version: 0.82
Working directory: 
/Users/miaocai/Dropbox/RprojectMiao/xgboost


In [2]:
# from sk.model_selection import train_test_split
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url)
train.head()
# y: survived

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url)
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


**split into train and test using scikit learn**

`
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, ySplit = train_test_split(x, y, test_size = 0.2, radom_state = 0)
`

In [5]:
X_columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 
            'Fare', 'Cabin', 'Embarked']
X_train = train[X_columns]
X_train = pd.get_dummies(X_train, columns = 
                         ['Pclass', 'Sex', 'Cabin', 'Embarked'])
y_train = train['Survived']
X_test = test[X_columns]

X_train.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Cabin_A10,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,38.0,1,0,71.2833,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,26.0,0,0,7.9250,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,35.0,1,0,53.1000,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,35.0,0,0,8.0500,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
xg_cl = xgb.XGBClassifier(objective = 'binary:logistic', n_estimator = 10, seed = 123)
xg_cl.fit(X_train, y_train)
preds = xg_cl.predict(X_train)
accuracy= float(np.sum(preds == y_train)/y_train.shape[0])
print("Prediction accuracy on train set:", np.round(accuracy*100, 2), "%")

Prediction accuracy on train set: 87.77 %
